In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset,DataLoader
import numpy as np
from scipy import interpolate
import random
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    #device = torch.device("cpu")
else:
    device = torch.device("cpu")


m=900#m+1 spacial nodes in total
Nnn=880+450+600
heat_up_number=10#number of warmback stages

os.chdir('/kaggle/input/multistagesinglefracture440')
XX_0= np.array(pd.read_csv('T_heatup_multistage_440.csv'))#take warmback stage temperature in wellbore
y_mm_0=np.array(pd.read_csv('position_multistage_singlefracture440.csv'))#take fracture labels along wellbore
real_position_0=np.array(pd.read_csv('singlefracture_multistage_realposition_440.csv'))#take real wellbore depth
XX_0=XX_0.reshape(440,m+1,heat_up_number)
XX_0=XX_0.transpose(0,2,1)
real_position_0=real_position_0.reshape(440,m+1,heat_up_number)
real_position_0=real_position_0.transpose(0,2,1)#(number_samples,heatup_stage,m=901)

os.chdir('/kaggle/input/multistagesinglefracturelater440')
XX_1= np.array(pd.read_csv('T_heatup_multistage_11.csv'))#(440*(m+1),heat_up_number)
y_mm_1=np.array(pd.read_csv('position_multistage_singlefracture111.csv'))#(440*heat_up_number,m+1)
real_position_1=np.array(pd.read_csv('singlefracture_multistage_realposition_11.csv'))#(440*(m+1),heat_up_number)
XX_1=XX_1.reshape(440,m+1,heat_up_number)
XX_1=XX_1.transpose(0,2,1)
real_position_1=real_position_1.reshape(440,m+1,heat_up_number)
real_position_1=real_position_1.transpose(0,2,1)
os.chdir('/kaggle/input/injection900')
T_inj_0=np.array(pd.read_csv('T_injection_train.csv')).reshape(880,m+1,22)[:440,:,0]
T_inj_1=np.array(pd.read_csv('T_injection_train.csv')).reshape(880,m+1,22)[440:,:,0]
Position_check=np.array(pd.read_csv('position_heatup_train.csv'))

os.chdir('/kaggle/input/3to5fractures')
y_mm_2=np.array(pd.read_csv('position_heatup_train_3to5_fracture.csv'))
T_inj_2=np.array(pd.read_csv('T_injection_train_3to5_fracture.csv')).reshape(450,m+1,22)[:,:,0]
os.chdir('/kaggle/input/multistage3ro5fracture450')
XX_2= np.array(pd.read_csv('3to5fracture_T_heatup_multistage450.csv'))
real_position_2=np.array(pd.read_csv('3to5fracture_multistage_realposition450.csv'))
XX_2=XX_2.reshape(450,m+1,heat_up_number)
XX_2=XX_2.transpose(0,2,1)
real_position_2=real_position_2.reshape(450,m+1,heat_up_number)
real_position_2=real_position_2.transpose(0,2,1)
os.chdir('/kaggle/input/2frac-multistage-variousflowrate-600-600')
index_frac1=np.array(pd.read_csv('fracture_L1_2frac_variousflowrate_0512.csv'))
y_mm_3=np.array(pd.read_csv('position_heatup_train_2frac_variousflowrate_0512.csv'))
T_inj_3=np.array(pd.read_csv('T_injection_train_2frac_variousflowrate_0512.csv')).reshape(600,m+1,22)[:,:,0]
frac_mm_singlestage1=np.array(pd.read_csv('fracflowrate_heatup_train_2frac_variousflowrate_0512.csv'))
XX_3= np.array(pd.read_csv('2fracture_variousflowrate_T_heatup_multistage_0512.csv'))
real_position_3=np.array(pd.read_csv('2fracture_variousflowrate_multistage_realposition_0512 .csv'))
Nnn2=600
heat_up_number=10
XX_3=XX_3.reshape(Nnn2,m+1,heat_up_number)
XX_3=XX_3.transpose(0,2,1)
real_position_3=real_position_3.reshape(Nnn2,m+1,heat_up_number)
real_position_3=real_position_3.transpose(0,2,1)
XX=np.zeros((Nnn,heat_up_number,m+1))
TT_inj=np.zeros((Nnn,m+1))
XX=torch.cat((torch.Tensor(XX_0),torch.Tensor(XX_1),torch.Tensor(XX_2),torch.Tensor(XX_3)),dim=0)
XX=XX.numpy()
XX=XX.reshape(Nnn*heat_up_number,m+1)
TT_inj=torch.cat((torch.Tensor(T_inj_0),torch.Tensor(T_inj_1),torch.Tensor(T_inj_2),torch.Tensor(T_inj_3)),dim=0)
TT_inj=TT_inj.numpy()
real_position=np.zeros((Nnn,heat_up_number,m+1))
real_position=torch.cat((torch.Tensor(real_position_0),torch.Tensor(real_position_1),torch.Tensor(real_position_2),torch.Tensor(real_position_3)),dim=0)                          
real_position=real_position.numpy()
real_position=real_position.reshape(Nnn*heat_up_number,m+1)
y_mm=np.zeros((Nnn,m+1))
y_mm_00=y_mm_0.reshape(440,heat_up_number,m+1)[:,0,:]
y_mm_11=y_mm_1.reshape(440,heat_up_number,m+1)[:,0,:]
y_mm=torch.cat((torch.Tensor(y_mm_00),torch.Tensor(y_mm_11),torch.Tensor(y_mm_2),torch.Tensor(y_mm_3)),dim=0)                        
y_mm=y_mm.numpy()
#interpolate
XX_new=np.zeros((Nnn*heat_up_number,m+1))
real_position_new=np.zeros((Nnn*heat_up_number,m+1))
TT_inj_new=np.zeros((Nnn,m+1))
for i in range(0,Nnn*heat_up_number):
    real_position_new[i,0:895]=np.arange(0,895)
    real_position_new[i,895]=894.5
    real_position_new[i,896]=895
    real_position_new[i,897]=895.5
    real_position_new[i,898]=896
    real_position_new[i,899]=896.5
    real_position_new[i,900]=897
    real_position_new[i,0]=1.05
for i in range(0,Nnn*heat_up_number):
    f = interpolate.interp1d(real_position[i,:], XX[i,:])
    XX_new[i,:]=f(real_position_new[i,:]) 

XX_new=XX_new.reshape(Nnn,heat_up_number,m+1)
#normalize temp, every sample for all heatup stages uses the same max and min
for i in range(Nnn):
    XX_new[i,:,:]=(XX_new[i,:,:]-np.min(XX_new[i,:,:]))/(np.max(XX_new[i,:,:])-np.min(XX_new[i,:,:]))
real_position_new=real_position_new.reshape(Nnn,heat_up_number,m+1)
#injection data interpolation
real_position=real_position.reshape(Nnn,heat_up_number,m+1)
for j in range(0,Nnn):
    ff = interpolate.interp1d(real_position[j,0,:], TT_inj[j,:])
    TT_inj_new[j,:]=ff(real_position_new[j,0,:]) 
for i in range(Nnn):
    TT_inj_new[i,:]=(TT_inj_new[i,:]-np.min(TT_inj_new[i,:]))/(np.max(TT_inj_new[i,:])-np.min(TT_inj_new[i,:]))    
    
XXXX=np.zeros((Nnn,heat_up_number+1,len(XX_new[0,0,:])))
XXXX[:,0,:]=TT_inj_new
XXXX[:,1:,:]=XX_new
N_validate=530
Index=random.sample(range(0,Nnn),Nnn-N_validate)
X_train=np.zeros((Nnn-N_validate,heat_up_number+1,len(XX_new[0,0,:])))
y_train=np.zeros((Nnn-N_validate,len(XX_new[0,0,:])))
X_test=[]
y_test=[]
for j in range(0,Nnn-N_validate):
    X_train[j]=XXXX[Index[j],:,:]
    y_train[j]=y_mm[Index[j],:]
aa=list(np.arange(0,Nnn))
cc=[x for x in aa if x in Index]
dd=[y for y in (aa+list(Index))if y not in cc]
for h in range(len(dd)):
    XXXX[dd[h],:,:]=XXXX[dd[h],:,:]
    X_test.append(XXXX[dd[h],:,:])
    y_test.append(y_mm[dd[h],:])
y_train_tensors = torch.LongTensor(y_train)
y_test_tensors = torch.LongTensor(np.array(y_test))
#add noise to input
noise_std_input = 0
X_train_noisy = X_train + noise_std_input * np.random.randn(*np.array(X_train).shape)
X_test_noisy=X_test + noise_std_input * np.random.randn(*np.array(X_test).shape)
X_train_tensors=torch.Tensor(X_train_noisy[:,:1,:])
X_test_tensors=torch.Tensor(X_test_noisy[:,:1,:])


class TemDataset(Dataset):
    def __init__(self,X_train_tensors,y_train_tensors):
        self.x=X_train_tensors
        self.y=y_train_tensors
        self.n_samples=self.x.shape[0]
    
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    
    def __len__(self):
        return self.n_samples

dataset=TemDataset(X_train_tensors,y_train_tensors)
dataloader=DataLoader(dataset=dataset,batch_size=256,shuffle=True)#444
print("Training Shape", X_train_tensors.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors.shape, y_test_tensors.shape)


In [ ]:
#single stage warmback
Num_feature=1
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length,embed_size=40*2,heads=1,drop_prob=0):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length
        self.embed_size=embed_size
        self.heads=heads
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,bidirectional=True,dropout=drop_prob) #lstm
        self.gru=nn.GRU(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True,dropout=drop_prob)
        self.multiattention=nn.MultiheadAttention(embed_size,heads,dropout=drop_prob)
        self.ln=nn.LayerNorm(normalized_shape=seq_length,eps=1.0e-5)
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)).to(device) #internal state
        x=self.ln(x.transpose(2,1))#add layer_norm
        x=x.transpose(2,1)
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        out = output
        return out    
class FCN(nn.Module):
    def __init__(self,c_in=Num_feature, c_out=1, layers=[128, 256, 128], kss=[7, 5, 3],embed_size1=128,embed_size2=256,heads=1,drop_prob=0):#c_in是feacture number
        super(FCN, self).__init__()
        assert len(layers) == len(kss)
        self.convblock1 = nn.Conv1d(c_in, layers[0], kss[0])
        self.convblock2 = nn.Conv1d(layers[0], layers[1], kss[1])
        self.convblock3 = nn.Conv1d(layers[1], layers[2], kss[2])
        self.gap=torch.nn.AdaptiveAvgPool1d((m+1))
        self.fc = nn.Linear((layers[2]+2*40), 10)
        self.fcc = nn.Linear(10,2)
        self.softmax=nn.Softmax(dim=1)
        self.sigmoid=nn.Sigmoid()
        self.relu=nn.ReLU()
        self.prelu=nn.PReLU()
        self.tanh=nn.Tanh()
        self.bn1=nn.BatchNorm1d(layers[0],eps=1e-03,momentum=0.99)
        self.bn2=nn.BatchNorm1d(layers[1],eps=1e-03,momentum=0.99)
        self.bn3=nn.BatchNorm1d(layers[2],eps=1e-03,momentum=0.99)
        self.dropout=nn.Dropout(p=0)
        self.lstm1 = LSTM1(num_classes=m+1, input_size=Num_feature, hidden_size=40, num_layers=2,seq_length=m+1).to(device)#input_size=feacture number
    def forward(self, x):
        x_fcn = self.convblock1(x)
        x_fcn=self.bn1(x_fcn)
        x_fcn=self.relu(x_fcn)
        x_fcn = self.convblock2(x_fcn)
        x_fcn=self.bn2(x_fcn)
        x_fcn=self.relu(x_fcn)
        x_fcn = self.convblock3(x_fcn)
        x_fcn=self.bn3(x_fcn)
        x_fcn=self.relu(x_fcn)
        x_fcn = self.gap(x_fcn)#size(N,C,m+1)
        x_fcn=x_fcn.transpose(1,2)
        ####LSTM
        x_lstm=x.transpose(1,2)
        x_lstm=self.lstm1(x_lstm)
        ###concat
        out_concat = torch.cat((x_lstm,x_fcn),2)
        out_concat=self.prelu(out_concat)
        out_concat=self.fc(out_concat)
        out_concat=self.prelu(out_concat)
        out_concat=self.fcc(out_concat)#size(N,m+1,2)
        outt=out_concat.transpose(1,2)#size(N,2,m+1)
        out_concat=self.prelu(outt)
        out_concat=self.softmax(outt)#calculate along dim=1
        return out_concat

In [ ]:
num_epochs = 800 #changale num of epochs
learning_rate = 0.005#changable learning rate
fcn = FCN().to(device) 
weight=torch.Tensor([1,100]).to(device)
criterion = torch.nn.CrossEntropyLoss(weight=weight)#weighted crossentropy loss
optimizer = torch.optim.Adam(fcn.parameters(), lr=learning_rate) 
scheduler=ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=50,
 verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
losses=[]
for epoch in range(num_epochs):
    for i,(inputs,labels) in enumerate(dataloader):  
        outputs = fcn.forward(inputs.to(device)) #forward pass
        loss = criterion(outputs, labels.to(device)) #forCrossEntropyLoss0411
        loss.backward() #calculates the loss of the loss function
        optimizer.step() #improve from loss, i.e backprop
        optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    if epoch % 100 == 0:
              losses.append(loss.item())
              print("Epoch: %d, loss: %1.5f" % (epoch, loss.item())) 

In [ ]:
#accuracy matrix:
train_predict1 = fcn(X_test_tensors.to(device))#forward pass
train_predict1=train_predict1.cpu()
data_predict = train_predict1.data.numpy() #numpy conversion
dataY_plot = y_test_tensors.data.numpy()
data_predict1=data_predict[:,1,:]
label_predicted=np.argwhere(np.around(data_predict1)==1)
label_real1=np.argwhere(dataY_plot==1)
sum_real=0
sum_predicted=0
for i in range (label_real1.shape[0]):
    for any in data_predict1[label_real1[i][0],label_real1[i][1]-4:label_real1[i][1]+5]:
        if any>=0.5:
            sum_real=sum_real+1
            break
for j in range(label_predicted.shape[0]):
    for any in dataY_plot[label_predicted[j][0],label_predicted[j][1]-4:label_predicted[j][1]+5]:
        if any==1:
            sum_predicted=sum_predicted+1
            break     
accuracy=2/(1/(sum_real/label_real1.shape[0])+1/(sum_predicted/label_predicted.shape[0]))

In [ ]:
#test a real case
#figure 6
m=900
os.chdir("/kaggle/input/modelfcn")
model=torch.load('single_warmback_prediction_0314_accuracy_99_76_utego_spike.pt')
os.chdir('/kaggle/input/ugeto-spike0121')
realposition=np.array(pd.read_csv('ugeto_spike.csv'))[0:1,:]#(10*582)
temp=np.array(pd.read_csv('ugeto_spike.csv'))[1:2,:]#(10*582)
XX_new11=np.zeros((1,m+1))
real_position_new11=np.zeros((1,m+1))
for i in range(0,1):
    real_position_new11[i,0:901]=np.arange(2450.582,2800,(2800-2451)/900)[0:901]
for i in range(0,1):
    f = interpolate.interp1d(realposition[i,:], temp[i,:])
    XX_new11[i,:]=f(real_position_new11[i,:])
XX_new11=(XX_new11-np.min(XX_new11))/(np.max(XX_new11)-np.min(XX_new11))
X_test_tensors2=torch.Tensor(np.zeros((1,901)))
X_test_tensors2[0,:]= Variable(torch.Tensor(XX_new11[0,:]))
X_test_tensors1=X_test_tensors2.unsqueeze(0)
train_predict2 = model(X_test_tensors1.to(device))#forward pass
train_predict2=train_predict2.cpu()
data_predict2 = train_predict2.data.numpy() #numpy conversion
#figure 6 in paper
os.chdir('/kaggle/input/ugeto-spike-frac-labels')
y_label=np.array(pd.read_csv('ugeto_spike_frac_labels.csv'))#(number_samples,m=901)
y_label[0,0]=2491.28
import matplotlib.patches as patches
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(0,9,2):
    rect = plt.Rectangle((y_label[0,i],0),y_label[0,i+1]-y_label[0,i],1, fill=True, color = "#C5C9C7",linewidth=1)
    ax.add_patch(rect)
rect = plt.Rectangle((y_label[0,10],0),y_label[0,11]-y_label[0,10],1, fill=True, color = "#C5C9C7",label="Labeled fracture zones")
ax.add_patch(rect)
ax.plot(real_position_new11[0,:],np.round(data_predict2[0,1,:]),"r-",linewidth=0.7,label="Predicted fractures")
ax2 = ax.twinx()
ax2.plot(real_position_new11[0,:],XX_new11[0,:],"b-",linewidth=1,label="Warmback temperature")
fig.legend(loc=6, bbox_to_anchor=(0.95,0.1))#, bbox_transform=ax.transAxes)
ax.set_ylabel("Fracture probability/[-]",color="r")
ax2.set_ylabel("Dimensionless wellbore temperature/[-]",color="b")
ax.set_xlabel("Depth/m")
plt.show()